In [1]:
import nest_asyncio
nest_asyncio.apply()

from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from utils import show_prompt, format_messages
from app.agents.prompts.deep_router_prompt import DEEP_ROUTER_PROMPT
from app.agents.prompts.system_rules import SYSTEM_RULES

In [2]:
from dotenv import load_dotenv
load_dotenv(".env", override=True)
%load_ext autoreload
%autoreload 2

In [3]:
show_prompt(DEEP_ROUTER_PROMPT)
show_prompt(SYSTEM_RULES)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  You are a Deep Routing Agent.                                                                                  │
│                                                                                                                 │
│  Your goal:                                                                                                     │
│  Decide which data source must be queried first to retrieve campaign requests                                   │
│  for further analysis.                                                                                          │
│                                                                                                                 │
│  Available tools:                                                                                               │
│  1. query_sql_campaigns                                                                                         │
│     - Use when the user provides a traffic source                                                               │
│     - Returns a list of campaign hashes                                                                         │
│                                                                                                                 │
│  2. query_mongo_requests                                                                                        │
│     - Use when the user provides a campaign hash                                                                │
│     - Use only after SQL if hashes come from traffic source                                                     │
│                                                                                                                 │
│  Rules:                                                                                                         │
│  - If the user provides a campaign hash, query MongoDB directly                                                 │
│  - If the user provides a traffic source, query SQL first                                                       │
│  - Never query MongoDB without at least one hash                                                                │
│  - Never query SQL if a hash is already provided                                                                │
│  - MongoDB queries must always be the final step                                                                │
│  - Do not analyze data, only retrieve it                                                                        │
│                                                                                                                 │
│  Think step by step and produce a clear execution plan.                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  Hard rules:                                                                                                    │
│  - Never invent hashes or traffic sources                                                                       │
│  - Never assume missing information                                                                             │
│  - If the user request is unclear, ask for clarification                                                        │
│  - Do not explain your reasoning to the user                                                                    │
│  - Do not perform analysis or conclusions                                                                       │
│  - Only plan tool execution                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Setting llm model

In [4]:
llm = ChatGoogleGenerativeAI(
      model="gemini-2.5-flash",
      temperature=0
)


### Importing the tools

In [5]:
from app.tools.mongo_tools import query_mongo_requests
from app.tools.sql_tools import query_sql_campaigns

tools = [query_sql_campaigns, query_mongo_requests]

c:\Users\Camille\Documents\TWR\deep_agents_twr\.venv\Lib\site-packages\motor\core.py:171: UserWarning: You appear to be connected to a DocumentDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/documentdb
  delegate = self.__delegate_class__(*args, **kwargs)


In [6]:
await query_sql_campaigns.ainvoke(
    {"traffic_source": "google"}
)

['0zq3byqr08',
 '9symhqv1fn',
 '23zekzhu19',
 'r90s02lj52',
 '31colzfd28',
 'fjcr6ghxk7',
 'hg23qnaqmd',
 'i5986uszqq',
 'uw0qfu4a1r',
 'w4rhsnmw02']

In [11]:
from app.tools.mongo_tools import query_mongo_requests

await query_mongo_requests.ainvoke(
    {"hash": "i5986uszqq"}
)

[{'_id': ObjectId('698a21c2024f8c26c45e5466'),
  'datetime': datetime.datetime(2026, 2, 9, 18, 4, 50, 219000),
  'decision': 'unsafe',
  'headers': '{"Host":"wcb.conexespsimples.online","X-Request-Id":"7fac77fd1599efa627b4ac5295ac1e6a","X-Real-Ip":"189.89.245.56","X-Forwarded-For":"189.89.245.56","X-Forwarded-Host":"wcb.conexespsimples.online","X-Forwarded-Port":"80","X-Forwarded-Proto":"http","X-Forwarded-Scheme":"http","X-Scheme":"http","X-Original-Forwarded-For":"189.89.245.56, 172.71.234.185","Forwarded":"for=172.71.234.185","Sec-Fetch-Dest":"empty","Referer":"https:\\/\\/conexespsimples.online\\/","Priority":"u=1, i","Accept-Language":"pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7","Cf-Ray":"9cb54a9c6eaeaf70-GRU","Accept-Encoding":"gzip, br","Sec-Ch-Ua-Platform":"\\"Android\\"","User-Agent":"Mozilla\\/5.0 (Linux; Android 10; K) AppleWebKit\\/537.36 (KHTML, like Gecko) Chrome\\/144.0.0.0 Mobile Safari\\/537.36","Sec-Ch-Ua":"\\"Not(A:Brand\\";v=\\"8\\", \\"Chromium\\";v=\\"144\\", \\"Google C